In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFaceHub, HuggingFaceEndpoint
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
import re
import textwrap


In [2]:
from langchain_community.document_loaders import PyPDFLoader

# Carregando o arquivo PDF
loader = PyPDFLoader('C:\\Users\\Graúna Motos\\Documents\\notebook\\Manual de Despesas ATUALIZADO (2) - Plan1.pdf')
documents = loader.load()

In [3]:
documents

[Document(metadata={'source': 'C:\\Users\\Graúna Motos\\Documents\\notebook\\Manual de Despesas ATUALIZADO (2) - Plan1.pdf', 'page': 0}, page_content='MANUAL DE DESPESAS\nITEM FINANCEIRO/OPERAÇÃO INTERNA/DESPESA TIPOS DE DESPESAS\nASSISTÊNCIA MÉDICA Saúde do trabalho (ASO, espirometria, etc.)\nExames, cirurgias, consultas, etc.\nSessões de psicoterapia\nCOMBUSTÍVEL VEÍCULOS FUNCIONÁRIOS/PRESTADORESCombustível para veículos de funcionários que não sejam vendedores e para veículos de prestadores de serviços, e que não seja veículos da Graúna.\nCOMBUSTÍVEIS MOTO 0KM Combustível para MOTOS 0KM\nCOMBUSTÍVEIS VEÍCULOS PRÓPRIOSCombustivel para Veículos da Graúna ( CAMINHÃO, S-10, etc...)\nCOMBUSTÍVEIS VENDEDORES Combustível para os Vendedores Graúna \nCONSERVAÇÃO DE BENS ESTRUTURAMaterial para reforma de prédio\nMão de obra ref. reforma do prédio\nServiço de pintura da loja\nFachada da loja\nConserto no forro\nTroca de vaso sanitário\nTudo que está relacionado a ESTRUTURA da loja \nFita Isola

In [4]:
def wrap_text_preserve_newlines(text, width=110):
    # split o texto em linhas baseado em caracteres de nova linha
    lines = text.split('\n')

    # envolve cada linha individualmente 
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # junta as linhas envolvidas de volta usando caracteres de nova linha
    wrapped_text = '\n'.join(wrapped_lines)

    

    return wrapped_text

In [5]:
print(wrap_text_preserve_newlines(str(documents[1])))

page_content='Despesas que envolvam móveis e utensílios (fogão, geladeira...) e imóvel (prédio, lotes, casas)
Manutenção caixa d´agua
Configuração IPAD
Confecção de capa p/caixa de som
Recarga de extintores
Limpeza de terrenos ou do quintal da loja
Conserto de fechaduras
Capa para tablet
Película para tablet
Divisórias p/prateleiras
Corte de árvores em terrenos da Graúna
Conserto do lava jato
Mangueira de incêndio
Serviço e material para fossa
Manutenção de Ar Condicionado
Instalação de Cortina de Ar
Manutenção e limpeza de tendas
Troca de materiais de sensores da loja
Limpeza de Placas Solares
Luzes de segurança
CURSOS/PALESTRAS/TREINAMENTOSMensalidade curso
Qualquer despesa com cursos e treinamentos, inclusive LANCHES E REFEIÇÕES
CUSTAS CARTÓRIO Reconhecimento de firma
Autenticação
Escritura
Procuração,  (que não seja para veículos seminovos.)
CUSTAS PROTESTOS Protesto de títulos
Cancelamento de protesto' metadata={'source': 'C:\\Users\\Graúna Motos\\Documents\\notebook\\Manual de
De

In [21]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000,  # Reduzido de 1000 para 500
    chunk_overlap=10,  # Adicionado um pequeno overlap para manter contexto
    separator="\n",
    length_function=len
)
docs = text_splitter.split_documents(documents)

# Verificar o número de chunks
print(f"Número de chunks: {len(docs)}")

# Opcional: visualizar o tamanho de cada chunk
for i, doc in enumerate(docs):
    print(f"\nChunk {i+1} - Tamanho: {len(doc.page_content)} caracteres")

Número de chunks: 13

Chunk 1 - Tamanho: 997 caracteres

Chunk 2 - Tamanho: 216 caracteres

Chunk 3 - Tamanho: 909 caracteres

Chunk 4 - Tamanho: 955 caracteres

Chunk 5 - Tamanho: 450 caracteres

Chunk 6 - Tamanho: 895 caracteres

Chunk 7 - Tamanho: 158 caracteres

Chunk 8 - Tamanho: 625 caracteres

Chunk 9 - Tamanho: 687 caracteres

Chunk 10 - Tamanho: 689 caracteres

Chunk 11 - Tamanho: 867 caracteres

Chunk 12 - Tamanho: 967 caracteres

Chunk 13 - Tamanho: 217 caracteres


In [16]:
print(docs[0])

page_content='MANUAL DE DESPESAS
ITEM FINANCEIRO/OPERAÇÃO INTERNA/DESPESA TIPOS DE DESPESAS
ASSISTÊNCIA MÉDICA Saúde do trabalho (ASO, espirometria, etc.)
Exames, cirurgias, consultas, etc.
Sessões de psicoterapia
COMBUSTÍVEL VEÍCULOS FUNCIONÁRIOS/PRESTADORESCombustível para veículos de funcionários que não sejam vendedores e para veículos de prestadores de serviços, e que não seja veículos da Graúna.
COMBUSTÍVEIS MOTO 0KM Combustível para MOTOS 0KM
COMBUSTÍVEIS VEÍCULOS PRÓPRIOSCombustivel para Veículos da Graúna ( CAMINHÃO, S-10, etc...)
COMBUSTÍVEIS VENDEDORES Combustível para os Vendedores Graúna 
CONSERVAÇÃO DE BENS ESTRUTURAMaterial para reforma de prédio
Mão de obra ref. reforma do prédio
Serviço de pintura da loja
Fachada da loja
Conserto no forro
Troca de vaso sanitário
Tudo que está relacionado a ESTRUTURA da loja 
Fita Isolante para usar na estrutura da loja
CONSERVAÇÃO DE BENS OUTROS Manutenção de imobilizado (impressoras, central de ar, computadores, etc.)
Lâmpadas
Interru

In [17]:
embeddings = HuggingFaceEmbeddings()

C:\Users\Graúna Motos\AppData\Local\Temp\ipykernel_16236\3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [22]:
db = FAISS.from_documents(docs, embeddings)

In [23]:
query = "ASSISTÊNCIA MÉDICA?"
docs = db.similarity_search(query)

In [24]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

MANUAL DE DESPESAS
ITEM FINANCEIRO/OPERAÇÃO INTERNA/DESPESA TIPOS DE DESPESAS
ASSISTÊNCIA MÉDICA Saúde do trabalho (ASO, espirometria, etc.)
Exames, cirurgias, consultas, etc.
Sessões de psicoterapia
COMBUSTÍVEL VEÍCULOS FUNCIONÁRIOS/PRESTADORESCombustível para veículos de funcionários que não sejam
vendedores e para veículos de prestadores de serviços, e que não seja veículos da Graúna.
COMBUSTÍVEIS MOTO 0KM Combustível para MOTOS 0KM
COMBUSTÍVEIS VEÍCULOS PRÓPRIOSCombustivel para Veículos da Graúna ( CAMINHÃO, S-10, etc...)
COMBUSTÍVEIS VENDEDORES Combustível para os Vendedores Graúna
CONSERVAÇÃO DE BENS ESTRUTURAMaterial para reforma de prédio
Mão de obra ref. reforma do prédio
Serviço de pintura da loja
Fachada da loja
Conserto no forro
Troca de vaso sanitário
Tudo que está relacionado a ESTRUTURA da loja
Fita Isolante para usar na estrutura da loja
CONSERVAÇÃO DE BENS OUTROS Manutenção de imobilizado (impressoras, central de ar, computadores, etc.)
Lâmpadas
Interruptores
Fios


In [25]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key="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",
    model_name="llama3-70b-8192"
)


In [26]:
chain = load_qa_chain(llm, chain_type="stuff")

C:\Users\Graúna Motos\AppData\Local\Temp\ipykernel_16236\1148363849.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [27]:
query = "ASSISTÊNCIA MÉDICA?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

C:\Users\Graúna Motos\AppData\Local\Temp\ipykernel_16236\3813654236.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=query)


'According to the manual, "ASSISTÊNCIA MÉDICA" refers to the following types of expenses:\n\n* Saúde do trabalho (ASO, espirometria, etc.)\n* Exames, cirurgias, consultas, etc.\n* Sessões de psicoterapia'